In [ ]:
import random             # Used to generated random Money
import operator           # USed for Dictionary operation
import itertools          # Used for Dictionary operation
import pandas as pd       # Used to keep a track of Buyers / Students money and bids left
import numpy as np        # Used to store the bidding matrix

In [ ]:
# Generating random price

money = []
for i in range(20):
  t = random.randint(200,400)
  money.append(t)

money.sort(reverse = True)
print(money)

[396, 396, 394, 376, 372, 362, 328, 315, 309, 296, 291, 287, 285, 266, 252, 238, 234, 227, 209, 205]


In [ ]:
# Creating a blank Series to keep a track of alloted electives

dict1 = {}
for i in range(20):
  z = i + 1
  dict1[z] = list()

alloted = pd.Series(dict1)
alloted

1     []
2     []
3     []
4     []
5     []
6     []
7     []
8     []
9     []
10    []
11    []
12    []
13    []
14    []
15    []
16    []
17    []
18    []
19    []
20    []
dtype: object

In [ ]:
# Creating a DF of buyer money and bids left

buyers = pd.DataFrame({'Buyers_Money' : money, 'Label': np.arange(1,21,1)})
buyers = buyers.set_index("Label")
buyers['Bid_left'] = 3
buyers

,Buyers_Money,Bid_left
Label,,
1,396,3
2,396,3
3,394,3
4,376,3
5,372,3
6,362,3
7,328,3
8,315,3
9,309,3


In [ ]:
# Initializig variables

min_price = 50
elective_offered = {1:0, 2:0, 3:0, 4:0, 5:0}

In [ ]:
def bidding(elective_offered):

  for i in range(20):

    # If zero bid left don't enter bidding
    if(buyers.iloc[i,1] == 0):
      continue

    print()
    print("*** Bidder ", (i+1),  " ***")

    while True:
      bids_on = input("Enter the product bids number: ")
      bids = bids_on.split(",")
      bids = [int(x) for x in bids]
      # Taking bid input

      # Checking if correct bid and corret number of bids are entered
      if(len(bids) == buyers.iloc[i,1]):
        flag = True
        for j in bids:
          if(j>0 and j<6):
            if(j not in elective_offered.keys()):
              print("Product ", j, " is not offered. Bid in other")
              flag = False
            if(elective_offered[j] == 15):
              print("Product ", j, " is SOLD")
              flag = False
            if(j in alloted[i+1]):
              print("You cannot bid on ",j, " as it is already alloted to you")
              flag = False
          else:
            print("Enter the products correctly")
            flag = False
        if(flag == True):
          break
      else:
        print("You have ",buyers.iloc[i,1], " bids left. Bid accordingly")


    # Buyer allocating money in the selected Electives
    buyer_money = buyers.iloc[i,0]
    counter = len(bids)
    for j in bids:
      while True:
        s = int(input("Enter the bid price for product " + str(j) + " : "))
        if (s >= min_price and s <= buyer_money and s>0):
          if ((buyer_money - s) >= ((counter-1) * 50)):
            buyer_money -= s
            break
          else:
            print("Available Money is Rs. ", buyer_money)
            print("Split your money wisely to participate in other products too")
        else:
          print("Available Money is Rs. ", buyer_money)
          print("Minimum value is ", min_price, '. Enter the bid correctly')
      counter -=1
      mat[j-1][i] = s


  return mat


In [ ]:
def best_prod(mat, elective_offered):

  # getting the best product
  temp2 = {}
  for i in elective_offered.keys():
    t = mat[i-1,:]

    non_zero = []
    for j in t:
      if (j > 0):
        non_zero.append(j)

    t = sorted(t, reverse=True)
    t = t[:len(non_zero)]

    sum_of_row = sum(non_zero)
    temp2[i] = sum_of_row

  temp2 = dict(sorted(temp2.items(), key=operator.itemgetter(1),reverse=True))
  temp3 = list(temp2.keys())[0]
  print("The best Product of this round is ", temp3)

  # Updating the Buyers List
  bid_amount = {}
  t = mat[temp3-1,:]
  count = 0
  for i in range(20):
    if(t[i] > 0):
      count += 1
    z = i + 1
    bid_amount[z] = t[i]
  bid_amount = dict(sorted(bid_amount.items(), key=operator.itemgetter(1),reverse=True))

  n = count
  already_alloted = elective_offered[temp3]
  if((already_alloted + n) > 15):
    n = 15 - already_alloted
  elective_offered[temp3] += n

  bid_amount = dict(itertools.islice(bid_amount.items(), n))

  for i in bid_amount.keys():
    alloted[i].append(temp3)

  top_bids = pd.Series(bid_amount)
  top_bids = top_bids.to_frame('Buyers_Money')
  top_bids["Bid_left"] = 1
  top_bids

  return buyers.subtract(top_bids, fill_value=0)

In [ ]:
# Final Bidding
c = 1
while True:
  print("------ Round ", (c)," ------")
  mat =  np.zeros((5, 20))
  mat = bidding(elective_offered)
  buyers = best_prod(mat,elective_offered)

  print("-----Updated Money-----")
  print(buyers)

  print("----- Alloted Products -----")
  print(alloted)

  print("----- Count of alloted Products -----")
  print(elective_offered)

  all_done = 0
  for j in range(20):
    if(buyers.iloc[j,1] == 0):
      all_done += 1

  if(all_done == 20):
    print("********** Bidding completed **********")
    break
371
  DF = pd.DataFrame(mat)

  # save the dataframe as a csv file
  file_name = "Data" + str(c) + ".csv"
  DF.to_csv(file_name)

  c += 1

------ Round  1  ------

*** Bidder  1  ***
Enter the product bids number: 1,2,6
Enter the products correctly
Enter the product bids number: 1,3
You have  3  bids left. Bid accordingly
Enter the product bids number: 1,2,3
Enter the bid price for product 1 : 200
Enter the bid price for product 2 : 100
Enter the bid price for product 3 : 96

*** Bidder  2  ***
Enter the product bids number: 2,3,4
Enter the bid price for product 2 : 200
Enter the bid price for product 3 : 150
Available Money is Rs.  196
Split your money wisely to participate in other products too
Enter the bid price for product 3 : 100
Enter the bid price for product 4 : 96

*** Bidder  3  ***
Enter the product bids number: 2,4,5
Enter the bid price for product 2 : 200
Enter the bid price for product 4 : 100
Enter the bid price for product 5 : 94

*** Bidder  4  ***
Enter the product bids number: 2,3,4
Enter the bid price for product 2 : 200
Enter the bid price for product 3 : 100
Enter the bid price for product 4 : 76

*

In [ ]:
# Checking if any elective is not offered

not_offered = []
for j in elective_offered.keys():
  if(elective_offered[j] < 5):
    not_offered.append(j)
    print("Product",not_offered, "is/are not offered")

if(len(not_offered) == 0 ):
  print("All offered")
else:
  for i in not_offered:
    del elective_offered[i]

Product [5] is/are not offered


In [ ]:
# Allocating different elective as per priority

for i in not_offered:
  for j in range(20):
    if(i in alloted[j+1]):
      print("Buyer", (j+1), "product",i,"is not offered. Please selcet other product.")
      while True:
        transfer = (int)(input("Which other product will you like to choose: "))
        if(elective_offered[transfer] < 15):
          alloted[j+1].remove(i)
          alloted[j+1].append(transfer)
          elective_offered[transfer] +=1
          break;
        else:
          print("The product is sold")


print(elective_offered)
print(alloted)

Buyer 4 product 5 is not offered. Please selcet other product.
Which other product will you like to choose: 2
The product is sold
Which other product will you like to choose: 3
The product is sold
Which other product will you like to choose: 4
Buyer 11 product 5 is not offered. Please selcet other product.
Which other product will you like to choose: 4
Buyer 17 product 5 is not offered. Please selcet other product.
Which other product will you like to choose: 4
Which other product will you like to choose: 4
Buyer 20 product 5 is not offered. Please selcet other product.
{1: 15, 2: 15, 3: 15, 4: 15}
1     [1, 2, 3]
2     [2, 3, 4]
3     [2, 3, 4]
4     [3, 4, 4]
5     [2, 4, 3]
6     [1, 2, 3]
7     [1, 3, 4]
8     [1, 2, 4]
9     [1, 2, 3]
10    [1, 3, 4]
11    [1, 2, 4]
12    [1, 2, 4]
13    [1, 2, 3]
14    [1, 4, 2]
15    [1, 3, 4]
16    [1, 2, 3]
17    [1, 2, 4]
18    [1, 2, 3]
19    [1, 2, 3]
20    [3, 4, 4]
dtype: object
